## Create Events from Plug Data

In [3]:
import joule
from joule.errors import ApiError

In [15]:
PLUG_STREAM = "/plugs/Kitchen Fridge"
EVENT_STREAM = "/plugs/events v2"
EVENT_NAME = "K Fridge"
MIN_WATTS = 100
REMOVE_EXISTING = True

In [21]:
node = joule.api.get_node()

plug_stream = await node.data_stream_get(PLUG_STREAM)
try:
    event_stream = await node.event_stream_get(EVENT_STREAM)
except ApiError:
    folder = "/".join(EVENT_STREAM.split('/')[:-1])
    name = EVENT_STREAM.split('/')[-1]
    event_stream = joule.api.EventStream(name=name, event_fields = {
        "name": "string",
        "average_power": "numeric",
        "peak_power": "numeric",
        "duration": "numeric"
    })
    event_stream = await node.event_stream_create(event_stream, folder)
await node.close()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7faaf43f8fd0>


In [22]:
node = joule.api.get_node()

if REMOVE_EXISTING:
    await node.event_stream_remove(event_stream, json_filter=[[['name','is',EVENT_NAME]]])
    

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7faaf43f8fd0>


TypeError: event_stream_remove() got an unexpected keyword argument 'json_filter'

In [ ]:
await node.close()